In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas
import time
import os
import xlwt
from openpyxl import load_workbook
from shutil import move
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = Options()
options.add_argument("--disable-notifications")
options.add_argument('headless')
chrome = webdriver.Chrome('./chromedriver')
amazon_allpage = []
data = []
tStart = time.time()
for i in range(1,3):
    chrome.get(url.format(i,i))
    soup = BeautifulSoup(chrome.page_source,'lxml')
    amazon_card=soup.find_all(class_="a-size-medium a-color-base a-text-normal")
    if len(amazon_card) == 0:
        print("找不到顯示卡標題")
        break
    data.append([ele.text for ele in amazon_card]) 
chrome.close()    
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

總共耗時:9.891093 秒


In [68]:

for adx,datas in enumerate(data):
    df = 'df'+str(adx) 
#     if datas[len(datas)-1]
    df = pd.DataFrame(datas,columns=['f'])
    print(len(datas)-1)
    if 'XFX' in datas[0]:
        print("你好")
    print(df['f'][0])
#     for aaa in range(0,len(datas)-1)


15
你好
XFX HDMI 顯卡
15
Sapphire 11293-03-40G Radeon NITRO+ RX 5700 XT 8GB GDDR6 雙 HDMI / 雙 DP OC (UEFI) PCIe 4.0 顯示卡


In [35]:
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))

In [36]:
now_date = datetime.strftime(datetime.now() , '%Y%m%d')
# 沒檔案的話則建立
output_file = now_date + "_amazon_graphics card.xlsx"
target_path = "E:\\python\\比價、受歡迎系統\\Amazon\\Amazon_graphics_card\\"
if os.path.exists(output_file) == False:
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book.create_sheet('default')  # 新增空sheet
else:  # 如果檔案存在，append
    book = load_workbook(output_file)
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book = book
# 假如excel已經有json的sheet_name則移除excel的sheet
if amazon_json['sheet_name'] in writer.book.sheetnames:
    writer.book.remove(writer.book[amazon_json['sheet_name']])


# 將資料寫入excel
for adx,datas in enumerate(data):
    df = 'df'+str(adx) 
    
    df = pd.DataFrame(datas)
    df.to_excel(writer, sheet_name=amazon_json['sheet_name'][adx], index=False)
    
if (len(writer.book.sheetnames) > 1):  # 若有其他sheet，刪除default sheet
    if 'default' in writer.book.sheetnames:
        writer.book.remove(writer.book['default'])    
writer.save()  


In [37]:
cwd = os.getcwd()
curr_file = cwd +"\\"+ output_file
target_file = target_path + output_file
try:
    move(curr_file, target_file)
    print(output_file + "\n移動檔案完畢.")
except Exception as e:
    print(output_file + "\n移動檔案失敗.")
    print(e.args)

20200907_amazon_graphics card.xlsx
移動檔案完畢.
